Cargamos las librerías que usaremos para generar la muestra de películas de Netflix conforme a las notas de referencia base:

https://heuristic-bhabha-ae33da.netlify.app/sistemas-de-recomendaci%C3%B3n-y-filtrado-colaborativo.html#ejemplo-datos-de-netflix

In [1]:
import pandas as pd
import numpy as np
import random

Comenzamos por cargar los datos de los títulos de películas y los de la base muestra de Netflix (Disponibles en: https://github.com/DorelyMS/proyecto-final-equipo5-mno-2020-1/tree/master/Sampling_Design/Reference_Files):

In [2]:
pelis_nombres=pd.read_csv('movies_title_fix.csv',names=["peli_id", "año", "nombre"])

In [3]:
pelis_nombres.head(10) #para ver cómo se encuentran los datos

,peli_id,año,nombre
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [4]:
df_muestra_netflix = pd.read_csv('dat_muestra_nflix.csv') #modificar ruta

In [5]:
df_muestra_netflix.head(10) #para ver cómo se encuentran los datos

,peli_id,usuario_id_orig,calif,fecha,usuario_id
0,1,2442,3,2004-04-14,1
1,1,1086807,3,2004-12-28,2
2,1,2165002,4,2004-04-06,3
3,1,1133214,4,2004-03-07,4
4,1,1537427,4,2004-03-29,5
5,1,525356,2,2004-07-11,6
6,1,1910569,4,2004-04-12,7
7,1,2421815,2,2004-02-26,8
8,1,2508819,3,2004-05-18,9
9,1,1342007,3,2004-07-16,10


In [6]:
df_muestra_netflix.shape #para saber el número de filas y columnas

(20968941, 5)

In [7]:
df_muestra_netflix_libmf=df_muestra_netflix[['usuario_id', 'peli_id', 'calif']] #seleccionamos únicamente las columnas que necesitamos para la librería y en el mismo orden

In [8]:
df_muestra_netflix_libmf.head(10)

,usuario_id,peli_id,calif
0,1,1,3
1,2,1,3
2,3,1,4
3,4,1,4
4,5,1,4
5,6,1,2
6,7,1,4
7,8,1,2
8,9,1,3
9,10,1,3


Ahora seleccionamos nuestra muestra de entrenamiento y de validación. Escogiendo una muestra de usuarios_id y de películas_id:

In [9]:
#empezando a armar la muestra de entrenamiento y validacion
random.seed(28882)
usuarios=pd.DataFrame(df_muestra_netflix_libmf['usuario_id'].unique(), columns=['usuario_id'])
valida_usuarios= usuarios.sample(frac=0.2)  
peliculas=pd.DataFrame(df_muestra_netflix_libmf['peli_id'].unique(), columns=['peli_id'])
valida_pelis=peliculas.sample(frac=0.2)

In [10]:
len(usuarios) #usuarios totales en movies_title_fix.csv

100000

In [11]:
len(valida_usuarios)

20000

In [12]:
len(peliculas) #películas totales en movies_title_fix.csv

17770

In [13]:
len(valida_pelis)

3554

In [14]:
#print(df_muestra_netflix_libmf_mini[df_muestra_netflix_libmf_mini['usuario_id'] == 29363])

In [15]:
#print(df_valida[df_valida['peli_id'] == 5582])

Y separamos calificaciones de entrenamiento y validación:

In [16]:
df_valida= pd.merge(df_muestra_netflix_libmf,valida_usuarios, on ='usuario_id', how='inner') #intersección entre df_muestra_netflix_libmf y valida_usuarios 

Verificamos que recuperamos los $20,000$ usuarios en validación:

In [17]:
len(df_valida['usuario_id'].unique())

20000

In [18]:
#muestra de validación
data_valida= pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner') #intersección entre df_valida y valida_pelis

Y notamos que perdimos 9 películas que no tenían calificaciones en validación (3,554-3,545), por lo que posteriormente se colocarán a entrenamiento:

In [19]:
len(data_valida['peli_id'].unique())

3552

In [20]:
data_valida.shape #dimension de la muestra de validación

(804465, 3)

Y vemos cómo quedaron los datos de la muestra de validación:

In [21]:
data_valida.head(10)

,usuario_id,peli_id,calif
0,5,205,4
1,2970,205,3
2,4158,205,4
3,4212,205,3
4,4546,205,4
5,5414,205,5
6,7984,205,3
7,8585,205,3
8,8647,205,1
9,8726,205,3


Exportamos la muestra de validación:

In [22]:
data_valida.to_csv(r'muestra_valida.csv', index = False)

Para obtener la muestra de entrenamiento, sacaremos los registros de df_muestra_netflix_libmf_mini que no estén en data_valida

In [ ]:
#union=pd.merge(df_muestra_netflix_libmf, data_valida, how='outer',on=['usuario_id', 'peli_id']) 
left= pd.merge(df_muestra_netflix_libmf, data_valida, how='left',on=['usuario_id', 'peli_id'])

In [43]:
data_entrena=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]

In [45]:
data_entrena.head(10)

,usuario_id,peli_id,calif_x
0,4536,1148,4
1,33379,14367,3
2,14173,11186,3
3,25154,8524,4
4,10125,10906,3
5,27070,11937,3
6,54043,494,5
7,7230,8643,4
8,36731,4141,4
9,10458,9616,4


In [46]:
data_entrena.shape

(201690, 3)

Exportamos la muestra de entrenamiento:

In [ ]:
data_entrena.to_csv(r'muestra_entrena.csv', index = False)